In [1]:
import sys
sys.path.append('/mnt/')
from finrl.models.baselines.ppo.agent import PPO
from finrl.models.baselines.ppo.policy import Policy
import json
from finrl.models.env import StockTradingEnv
from finrl.data.generate import stock_trade_data_generate
from finrl.models.constants import *
import torch
from torch import nn
import numpy as np
from finrl.models.utils import data_split

/root/miniconda3/envs/myconda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 导入参数
path = '/mnt/finrl/models/baselines/ppo/kwargs.json'
with open(path,'r',encoding='utf-8') as f:
    config = json.load(f)
print(config)

{'env_trade_kwargs': {'stock_dim': None, 'hmax': 100, 'initial_amount': 1000000, 'num_stock_shares': None, 'buy_cost_pct': None, 'sell_cost_pct': None, 'reward_scaling': 0.0001, 'state_space': None, 'action_space': None, 'tech_indicator_list': ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma'], 'turbulence_threshold': None, 'risk_indicator_col': 'turbulence', 'reward_aliase': 'asset_diff', 'cash_norm_factor': 1e-06, 'num_share_norm_factor': None, 'if_price_norm': None, 'if_indicator_norm': None, 'if_num_share_norm': None, 'max_price': None, 'min_price': None}, 'env_portfolio_kwargs': {'stock_dim': None, 'hmax': 100, 'initial_amount': 1000000, 'buy_cost_pct': None, 'sell_cost_pct': None, 'reward_scaling': 0.0001, 'state_space': None, 'action_space': None, 'tech_indicator_list': ['macd', 'rsi_30', 'cci_30', 'dx_30'], 'turbulence_threshold': None, 'lookback': 252, 'reward_aliase': 'asset_diff'}, 'policy_kwargs': {'state_dim': None, 'action_dim': No

In [3]:
# 数据路径
data_dir = '/mnt/finrl/data/csv/DOW_30/'
vix_data_dir = '/mnt/finrl/data/csv/^VIX/'
dji_dir = '/mnt/finrl/data/csv/DJI/DJI.csv'  # .csv

# 设置日期
TRAIN_START_DATE = '2010-01-01'
TRAIN_END_DATE = '2021-12-31'
VALIDATE_START_DATE = '2022-01-01'
VALIDATE_END_DATE = '2022-12-31'
TEST_START_DATE = '2023-01-01'
TEST_END_DATE = '2024-01-30'

# 获取数据
df = stock_trade_data_generate(
    data_dir=data_dir,
    start_date=TRAIN_START_DATE,
    end_date=TEST_END_DATE,
    use_technical_indicator=True,
    use_turbulence=True,
    user_defined_feature=False,
    tech_indicator_list=INDICATORS,
    use_vix=True,
    vix_data_dir=vix_data_dir,
    dji_dir=dji_dir,
)

# 数据划分
df_train = data_split(df,TRAIN_START_DATE,TRAIN_END_DATE)
df_validation = data_split(df,VALIDATE_START_DATE,VALIDATE_END_DATE)
df_test = data_split(df,TEST_START_DATE,TEST_END_DATE)
df_train_validation = data_split(df,TRAIN_START_DATE,VALIDATE_END_DATE)

print(TRAIN_START_DATE,TRAIN_END_DATE,VALIDATE_START_DATE,VALIDATE_END_DATE,TEST_START_DATE,TEST_END_DATE)

# 计算预警机制阈值
validation_risk_indicator = df_validation.drop_duplicates(subset = ['date'])
validation_vix_threshold = validation_risk_indicator.vix.quantile(0.996)
test_risk_indicator = df_train_validation.drop_duplicates(subset = ['date'])  # 不能用到测试集数据
test_vix_threshold = test_risk_indicator.vix.quantile(0.996)

# 进行归一化

# 进行价格归一化
if_price_norm = True
min_price,max_price = [],[]
if if_price_norm:
    print('正在进行价格归一化')
    tic_list = df_train_validation.tic.unique().tolist()
    min_price,max_price = [],[]
    for tic in tic_list:
        df1 = df_train_validation[df_train_validation['tic']==tic]
        min_price.append(df1.close.min())
        max_price.append(df1.close.max())

    column = df_train.columns.get_loc('close')
    for i in range(df_train.shape[0]): # 遍历每行
        tic = df_train.iloc[i]['tic'] # 找到此行对应的股票
        index=tic_list.index(tic)
        df_train.iat[i,column] = (df_train.iat[i,column] - min_price[index]) / (max_price[index] - min_price[index])

    column = df_validation.columns.get_loc('close')
    for i in range(df_validation.shape[0]): # 遍历每行
        tic = df_validation.iloc[i]['tic'] # 找到此行对应的股票
        index=tic_list.index(tic)
        df_validation.iat[i,column] = (df_validation.iat[i,column] - min_price[index]) / (max_price[index] - min_price[index])

    column = df_train_validation.columns.get_loc('close')
    for i in range(df_train_validation.shape[0]): # 遍历每行
        tic = df_train_validation.iloc[i]['tic'] # 找到此行对应的股票
        index=tic_list.index(tic)
        df_train_validation.iat[i,column] = (df_train_validation.iat[i,column] - min_price[index]) / (max_price[index] - min_price[index])

    column = df_test.columns.get_loc('close')
    for i in range(df_test.shape[0]): # 遍历每行
        tic = df_test.iloc[i]['tic'] # 找到此行对应的股票
        index=tic_list.index(tic)
        df_test.iat[i,column] = (df_test.iat[i,column] - min_price[index]) / (max_price[index] - min_price[index])

# 进行技术指标归一化。
if_indicator_norm = False
min_indicator,max_indicator = [],[]
if if_indicator_norm:
    print('正在进行技术指标归一化')
    tic_list = df_train_validation.tic.unique().tolist()
    min_indicator,max_indicator = [],[]

    for indicator in INDICATORS:
        df1 = df_train_validation[['tic',indicator]]
        for tic in tic_list:
            minimum = df1[df1.tic==tic][indicator].min()
            maximum = df1[df1.tic==tic][indicator].max()
            min_indicator.append(minimum)
            max_indicator.append(maximum)
    
    for i in range(df_train.shape[0]):
        tic_ = df_train.iloc[i]['tic']
        tic_index = tic_list.index(tic_)
        for indicator in INDICATORS:
            indicator_index = INDICATORS.index(indicator)
            indicator_column_index = df_train.columns.get_loc(indicator)
            df_train.iat[i,indicator_column_index] = (df_train.iat[i,indicator_column_index] - min_indicator[indicator_index*len(tic_list)+tic_index]) / (max_indicator[indicator_index*len(tic_list)+tic_index] - min_indicator[indicator_index*len(tic_list)+tic_index])

    for i in range(df_train_validation.shape[0]):
        tic_ = df_train_validation.iloc[i]['tic']
        tic_index = tic_list.index(tic_)
        for indicator in INDICATORS:
            indicator_index = INDICATORS.index(indicator)
            indicator_column_index = df_train_validation.columns.get_loc(indicator)
            df_train_validation.iat[i,indicator_column_index] = (df_train_validation.iat[i,indicator_column_index] - min_indicator[indicator_index*len(tic_list)+tic_index]) / (max_indicator[indicator_index*len(tic_list)+tic_index] - min_indicator[indicator_index*len(tic_list)+tic_index])

    for i in range(df_validation.shape[0]):
        tic_ = df_validation.iloc[i]['tic']
        tic_index = tic_list.index(tic_)
        for indicator in INDICATORS:
            indicator_index = INDICATORS.index(indicator)
            indicator_column_index = df_validation.columns.get_loc(indicator)
            df_validation.iat[i,indicator_column_index] = (df_validation.iat[i,indicator_column_index] - min_indicator[indicator_index*len(tic_list)+tic_index]) / (max_indicator[indicator_index*len(tic_list)+tic_index] - min_indicator[indicator_index*len(tic_list)+tic_index])


    for i in range(df_test.shape[0]):
        tic_ = df_test.iloc[i]['tic']
        tic_index = tic_list.index(tic_)
        for indicator in INDICATORS:
            indicator_index = INDICATORS.index(indicator)
            indicator_column_index = df_test.columns.get_loc(indicator)
            df_test.iat[i,indicator_column_index] = (df_test.iat[i,indicator_column_index] - min_indicator[indicator_index*len(tic_list)+tic_index]) / (max_indicator[indicator_index*len(tic_list)+tic_index] - min_indicator[indicator_index*len(tic_list)+tic_index])

# 持股数归一化
if_num_share_norm=False

# 创建环境
STOCK_DIM = len(df.tic.unique())
ACTION_DIM = STOCK_DIM  
STATE_DIM = 1 + ACTION_DIM * (1+1) + ACTION_DIM*8
print(STOCK_DIM,ACTION_DIM,STATE_DIM)


env_trade_kwargs = config['env_trade_kwargs']
env_trade_kwargs.update({
    'num_stock_shares':STOCK_DIM*[0],
    'stock_dim':STOCK_DIM,
    'state_space':STATE_DIM,
    'action_space':STOCK_DIM,
    'buy_cost_pct':[0.001]*ACTION_DIM,
    'sell_cost_pct':[0.001]*ACTION_DIM,
    'reward_aliase':'asset_diff',
    'reward_scaling':1e-4,
    'tech_indicator_list':INDICATORS,
    'cash_norm_factor':1e-6,  # cash缩放 
    'num_share_norm_factor':1,
    'if_price_norm':if_price_norm,
    'if_indicator_norm':if_indicator_norm,
    'if_num_share_norm':if_num_share_norm,
    'max_price':max_price,
    'min_price':min_price,
})

# 设置需要的属性，设置预警机制
env_train = StockTradingEnv(df_train, **env_trade_kwargs)
env_train.DATE_START,env_train.DATE_END = TRAIN_START_DATE,TRAIN_END_DATE
env_train_validation = StockTradingEnv(df_train_validation, **env_trade_kwargs)
env_train_validation.DATE_START,env_train_validation.DATE_END = TRAIN_START_DATE,VALIDATE_END_DATE

env_validation = StockTradingEnv(df_validation, **env_trade_kwargs)
env_validation.DATE_START,env_validation.DATE_END = VALIDATE_START_DATE,VALIDATE_END_DATE
env_validation.risk_indicator_col = 'vix'    # 设置预警
env_validation.turbulence_threshold = validation_vix_threshold

env_test = StockTradingEnv(df_test, **env_trade_kwargs)  
env_test.DATE_START,env_test.DATE_END = TEST_START_DATE,TEST_END_DATE
env_test.risk_indicator_col = 'vix'           # 设置预警
env_test.turbulence_threshold = test_vix_threshold

print(env_trade_kwargs)

/mnt/finrl/data/yahoo.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)
/mnt/finrl/data/yahoo.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)
/mnt/finrl/data/yahoo.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)
/mnt/finrl/data/yahoo.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)
/mnt/finrl/data/yahoo.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)
/mnt/finrl

Shape of DataFrame:  (103943, 8)


/mnt/finrl/data/feature.py:133: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/mnt/finrl/data/feature.py:133: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/mnt/finrl/data/feature.py:133: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/mnt/finrl/data/feature.py:133: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/mnt/finrl/data/feature.py:133: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indica

Successfully added technical indicators
Shape of DataFrame:  (3542, 8)
Successfully added vix
Successfully added turbulence index
2010-01-01 2021-12-31 2022-01-01 2022-12-31 2023-01-01 2024-01-30
正在进行价格归一化
29 29 291
{'stock_dim': 29, 'hmax': 100, 'initial_amount': 1000000, 'num_stock_shares': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'buy_cost_pct': [0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001], 'sell_cost_pct': [0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001], 'reward_scaling': 0.0001, 'state_space': 291, 'action_space': 29, 'tech_indicator_list': ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60

In [4]:
policy_kwargs = config['policy_kwargs']
policy_kwargs.update(
    {
        'state_dim': STATE_DIM,
        'action_dim': ACTION_DIM,
        'activation_fn': nn.ReLU,
        'log_std_init': 0.0
    }
)

agent_kwargs = config['agent_kwargs']
agent_kwargs.update(
    {
        'env_train':env_train,
        'env_validation':env_validation,
        'env_test':env_test,
        'total_timesteps': 30000,  # 大约10幕
        'n_rollout_steps': 256,   
        'n_updates':100, 
        'max_grad_norm': 0.5,  ### 检测它的效果。
        'lambda_coef':0.95,
        'clip_range':0.2,
        'policy_lr':3e-4,
        'ent_coef':0,
        'value_coef':0.5,
        'policy_kwargs':policy_kwargs,
        'batch_size':64,
        'gamma':0.99,
    }
)

In [6]:
for i in range(1,11):
    print(f'-------------------------------------------这是第{i}次训练---------------------------------------------')
    agent_kwargs['train_time'] = str(i)
    agent = PPO(**agent_kwargs)
    agent.train()